In [1]:
import numpy as np

In [9]:
data = open('intro.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('Data has %d characters, %d unique' %(data_size, vocab_size))

char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i,ch in enumerate(chars)}

#hyperparameters

hidden_size = 100
learning_rate = 1e-1
seq_length = 25

#model parameters

Wxh = np.random.randn(hidden_size, vocab_size)*0.01
Whh = np.random.randn(hidden_size, hidden_size)*0.01
Why = np.random.randn(vocab_size, hidden_size)*0.01
bh = np.zeros((hidden_size,1))
by = np.zeros((vocab_size,1))

Data has 5 characters, 4 unique


In [10]:
def Lossfun(inputs, targets, hprev):
    
    xs, hs, ys, ps = {},{},{},{}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # Forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t])/np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t],0])
     
    # Backward pass
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        dy = np.copy(ys[t])
        dy[targets[t]] -= 1
        
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = ((1 - hs[t])*hs[t]) * dh
        
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        
        dhnext += np.dot(Whh.T, dhraw)
        
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]  


def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y)/np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p = p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1 
        ixes.append(ix)
    return ixes

n,p = 0,0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

while True:
    
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))
        p = 0
        
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [ char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    
    if n%100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----'%(txt, ))
    
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = Lossfun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter %d, loss: %f'% (n, smooth_loss))
              
# Parameter Update with Adagrad              
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

        p += seq_length # move data pointer
        n += 1 # iteration counter 

----
 Heell 
----
iter 0, loss: 4.158885
----
 oeeHe 
----
iter 100, loss: 4.139738
----
 olHHH 
----
iter 200, loss: 4.102519
----
 elHHl 
----
iter 300, loss: 4.065482
----
 eleoe 
----
iter 400, loss: 4.029142
----
 ollHl 
----
iter 500, loss: 3.993519
----
 eeoHl 
----
iter 600, loss: 3.958602
----
 eHHHe 
----
iter 700, loss: 3.924376
----
 HlooH 
----
iter 800, loss: 3.890829
----
 Hoeoe 
----
iter 900, loss: 3.857946
----
 eleeo 
----
iter 1000, loss: 3.825714
----
 eeoHe 
----
iter 1100, loss: 3.794121
----
 lHlHl 
----
iter 1200, loss: 3.763154
----
 elllo 
----
iter 1300, loss: 3.732800
----
 oleol 
----
iter 1400, loss: 3.703048
----
 oleHl 
----
iter 1500, loss: 3.673885
----
 olell 
----
iter 1600, loss: 3.645300
----
 ellll 
----
iter 1700, loss: 3.617281
----
 elllH 
----
iter 1800, loss: 3.589818
----
 oeeel 
----
iter 1900, loss: 3.562898
----
 ellHl 
----
iter 2000, loss: 3.536512
----
 elHee 
----
iter 2100, loss: 3.510648
----
 olHll 
----
iter 2200, loss: 3.485297


----
 oeoHl 
----
iter 19600, loss: 2.269579
----
 olool 
----
iter 19700, loss: 2.268815
----
 oolHe 
----
iter 19800, loss: 2.268066
----
 Hlooo 
----
iter 19900, loss: 2.267332
----
 leeel 
----
iter 20000, loss: 2.266612
----
 eHeel 
----
iter 20100, loss: 2.265907
----
 eoHee 
----
iter 20200, loss: 2.265215
----
 eHeeH 
----
iter 20300, loss: 2.264538
----
 olooo 
----
iter 20400, loss: 2.263873
----
 lHoHl 
----
iter 20500, loss: 2.263222
----
 ooHol 
----
iter 20600, loss: 2.262584
----
 eollo 
----
iter 20700, loss: 2.261958
----
 eleel 
----
iter 20800, loss: 2.261345
----
 oolee 
----
iter 20900, loss: 2.260744
----
 HHloo 
----
iter 21000, loss: 2.260155
----
 loeol 
----
iter 21100, loss: 2.259577
----
 elHoe 
----
iter 21200, loss: 2.259011
----
 eolHl 
----
iter 21300, loss: 2.258456
----
 loleH 
----
iter 21400, loss: 2.257913
----
 lHooH 
----
iter 21500, loss: 2.257380
----
 elHoe 
----
iter 21600, loss: 2.256857
----
 eHlHo 
----
iter 21700, loss: 2.256345
----
 HHoe

----
 eHooo 
----
iter 38200, loss: 2.231938
----
 eHlll 
----
iter 38300, loss: 2.231920
----
 HHeoH 
----
iter 38400, loss: 2.231902
----
 elole 
----
iter 38500, loss: 2.231884
----
 elooo 
----
iter 38600, loss: 2.231866
----
 eoleo 
----
iter 38700, loss: 2.231849
----
 eolle 
----
iter 38800, loss: 2.231833
----
 elHlo 
----
iter 38900, loss: 2.231816
----
 ellee 
----
iter 39000, loss: 2.231800
----
 Heele 
----
iter 39100, loss: 2.231784
----
 eeHoH 
----
iter 39200, loss: 2.231769
----
 elloo 
----
iter 39300, loss: 2.231754
----
 llolo 
----
iter 39400, loss: 2.231739
----
 eelHo 
----
iter 39500, loss: 2.231724
----
 eleol 
----
iter 39600, loss: 2.231710
----
 Hllol 
----
iter 39700, loss: 2.231696
----
 oleHe 
----
iter 39800, loss: 2.231683
----
 eelll 
----
iter 39900, loss: 2.231669
----
 oelee 
----
iter 40000, loss: 2.231656
----
 eleHH 
----
iter 40100, loss: 2.231643
----
 Hleee 
----
iter 40200, loss: 2.231630
----
 eHoee 
----
iter 40300, loss: 2.231618
----
 lelH

----
 elloe 
----
iter 57000, loss: 2.231027
----
 ellHH 
----
iter 57100, loss: 2.231026
----
 lHele 
----
iter 57200, loss: 2.231026
----
 eeleo 
----
iter 57300, loss: 2.231026
----
 lHHel 
----
iter 57400, loss: 2.231025
----
 oleel 
----
iter 57500, loss: 2.231025
----
 eelol 
----
iter 57600, loss: 2.231024
----
 Hoole 
----
iter 57700, loss: 2.231024
----
 Hlleo 
----
iter 57800, loss: 2.231024
----
 HelHH 
----
iter 57900, loss: 2.231023
----
 oolHl 
----
iter 58000, loss: 2.231023
----
 elell 
----
iter 58100, loss: 2.231023
----
 eHoee 
----
iter 58200, loss: 2.231022
----
 lloel 
----
iter 58300, loss: 2.231022
----
 HHlll 
----
iter 58400, loss: 2.231022
----
 eHlHo 
----
iter 58500, loss: 2.231021
----
 olHel 
----
iter 58600, loss: 2.231021
----
 ellee 
----
iter 58700, loss: 2.231021
----
 ellHl 
----
iter 58800, loss: 2.231020
----
 oeoHl 
----
iter 58900, loss: 2.231020
----
 ellol 
----
iter 59000, loss: 2.231020
----
 eloeo 
----
iter 59100, loss: 2.231019
----
 ellH

iter 75200, loss: 2.231006
----
 lloHo 
----
iter 75300, loss: 2.231006
----
 ooHHe 
----
iter 75400, loss: 2.231006
----
 eleHH 
----
iter 75500, loss: 2.231006
----
 oeoel 
----
iter 75600, loss: 2.231006
----
 oHoHl 
----
iter 75700, loss: 2.231006
----
 oHlHl 
----
iter 75800, loss: 2.231006
----
 lleel 
----
iter 75900, loss: 2.231006
----
 HHole 
----
iter 76000, loss: 2.231006
----
 lHoHl 
----
iter 76100, loss: 2.231006
----
 eelll 
----
iter 76200, loss: 2.231006
----
 oeell 
----
iter 76300, loss: 2.231006
----
 oeoll 
----
iter 76400, loss: 2.231006
----
 oolel 
----
iter 76500, loss: 2.231006
----
 HlHHo 
----
iter 76600, loss: 2.231006
----
 eolel 
----
iter 76700, loss: 2.231006
----
 Hooee 
----
iter 76800, loss: 2.231006
----
 HeoeH 
----
iter 76900, loss: 2.231006
----
 eleoH 
----
iter 77000, loss: 2.231006
----
 lllle 
----
iter 77100, loss: 2.231006
----
 ellHe 
----
iter 77200, loss: 2.231006
----
 lllol 
----
iter 77300, loss: 2.231006
----
 HHlll 
----
iter 77400

...

----
 ollol 
----
iter 459700, loss: 2.231005
----
 eeloe 
----
iter 459800, loss: 2.231005
----
 olHHl 
----
iter 459900, loss: 2.231005
----
 eHlel 
----
iter 460000, loss: 2.231005
----
 lHHoe 
----
iter 460100, loss: 2.231005
----
 olHol 
----
iter 460200, loss: 2.231005
----
 llolH 
----
iter 460300, loss: 2.231005
----
 HHHHe 
----
iter 460400, loss: 2.231005
----
 lllel 
----
iter 460500, loss: 2.231005
----
 ellol 
----
iter 460600, loss: 2.231005
----
 elell 
----
iter 460700, loss: 2.231005
----
 HHlol 
----
iter 460800, loss: 2.231005
----
 oelHe 
----
iter 460900, loss: 2.231005
----
 lHHeH 
----
iter 461000, loss: 2.231005
----
 HollH 
----
iter 461100, loss: 2.231005
----
 Hloel 
----
iter 461200, loss: 2.231005
----
 olHoe 
----
iter 461300, loss: 2.231005
----
 Hlooo 
----
iter 461400, loss: 2.231005
----
 eleeH 
----
iter 461500, loss: 2.231005
----
 lllll 
----
iter 461600, loss: 2.231005
----
 eHHeo 
----
iter 461700, loss: 2.231005
----
 ellll 
----
iter 461800, los

----
 eollo 
----
iter 478300, loss: 2.231005
----
 lHeHl 
----
iter 478400, loss: 2.231005
----
 eolel 
----
iter 478500, loss: 2.231005
----
 HlHlo 
----
iter 478600, loss: 2.231005
----
 ellol 
----
iter 478700, loss: 2.231005
----
 HHool 
----
iter 478800, loss: 2.231005
----
 elolo 
----
iter 478900, loss: 2.231005
----
 eHHeo 
----
iter 479000, loss: 2.231005
----
 HelHl 
----
iter 479100, loss: 2.231005
----
 ellHe 
----
iter 479200, loss: 2.231005
----
 eHeee 
----
iter 479300, loss: 2.231005
----
 oHlll 
----
iter 479400, loss: 2.231005
----
 olool 
----
iter 479500, loss: 2.231005
----
 eeloe 
----
iter 479600, loss: 2.231005
----
 elloH 
----
iter 479700, loss: 2.231005
----
 eleol 
----
iter 479800, loss: 2.231005
----
 ellel 
----
iter 479900, loss: 2.231005
----
 oolll 
----
iter 480000, loss: 2.231005
----
 eHHoo 
----
iter 480100, loss: 2.231005
----
 ellel 
----
iter 480200, loss: 2.231005
----
 eolHl 
----
iter 480300, loss: 2.231005
----
 eeeHl 
----
iter 480400, los

----
 leloe 
----
iter 496500, loss: 2.231005
----
 ellel 
----
iter 496600, loss: 2.231005
----
 ellee 
----
iter 496700, loss: 2.231005
----
 ellel 
----
iter 496800, loss: 2.231005
----
 ollHH 
----
iter 496900, loss: 2.231005
----
 HHlol 
----
iter 497000, loss: 2.231005
----
 looel 
----
iter 497100, loss: 2.231005
----
 loool 
----
iter 497200, loss: 2.231005
----
 eHoHl 
----
iter 497300, loss: 2.231005
----
 eoloo 
----
iter 497400, loss: 2.231005
----
 eolHl 
----
iter 497500, loss: 2.231005
----
 eloee 
----
iter 497600, loss: 2.231005
----
 ooeoe 
----
iter 497700, loss: 2.231005
----
 leHlo 
----
iter 497800, loss: 2.231005
----
 ollle 
----
iter 497900, loss: 2.231005
----
 eolel 
----
iter 498000, loss: 2.231005
----
 oHlol 
----
iter 498100, loss: 2.231005
----
 oeHlH 
----
iter 498200, loss: 2.231005
----
 olHee 
----
iter 498300, loss: 2.231005
----
 lellH 
----
iter 498400, loss: 2.231005
----
 Hllll 
----
iter 498500, loss: 2.231005
----
 eHlle 
----
iter 498600, los

----
 eHeee 
----
iter 515200, loss: 2.231005
----
 eleHl 
----
iter 515300, loss: 2.231005
----
 eleel 
----
iter 515400, loss: 2.231005
----
 Holel 
----
iter 515500, loss: 2.231005
----
 ellHl 
----
iter 515600, loss: 2.231005
----
 eleol 
----
iter 515700, loss: 2.231005
----
 Hllel 
----
iter 515800, loss: 2.231005
----
 elHHe 
----
iter 515900, loss: 2.231005
----
 loelo 
----
iter 516000, loss: 2.231005
----
 HelHo 
----
iter 516100, loss: 2.231005
----
 HHeee 
----
iter 516200, loss: 2.231005
----
 ellee 
----
iter 516300, loss: 2.231005
----
 Helee 
----
iter 516400, loss: 2.231005
----
 Hllll 
----
iter 516500, loss: 2.231005
----
 eelee 
----
iter 516600, loss: 2.231005


KeyboardInterrupt: 